In [1]:
import torch
import torchvision
import torchvision.transforms as transforms 
import torch.nn as nn
import torch.utils.data as data
import numpy as np

class_labels = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'
}

fashion_trainset = torchvision.datasets.FashionMNIST(root="./data", train=True, download=True, transform=transforms.ToTensor())
fashion_loader = data.DataLoader(fashion_trainset, batch_size=32, shuffle=True)

mnist_trainset = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=transforms.ToTensor())
mnist_loader = data.DataLoader(mnist_trainset, batch_size=32, shuffle=True)

images, labels = (fashion_loader.dataset.data, fashion_loader.dataset.targets)

m_images, m_labels = (mnist_loader.dataset.data, mnist_loader.dataset.targets)

res_images = torch.Tensor(m_images.shape[0], 28, 84)
res_labels = torch.Tensor(m_labels.shape[0])

for i in range(m_images.shape[0]):
    rand_idx1 = np.random.randint(0, len(labels))
    rand_idx2 = np.random.randint(0, len(labels))
    res_images[i] = torch.cat((images[rand_idx1],m_images[0], images[rand_idx2]), dim=1)
    if m_labels[0] % 2 == 0: 
        res_labels[i] = labels[rand_idx1].item() 
    else: 
        res_labels[i] = labels[rand_idx2].item()


In [2]:
dataset = data.TensorDataset(res_images, res_labels)
test_size = 0.2 
num_test_samples = int(len(dataset) * test_size)
num_train_samples = len(dataset) - num_test_samples
train_dataset, test_dataset = data.random_split(dataset, [num_train_samples, num_test_samples])

batch_size = 128 
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3,padding=1)
        self.conv2 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,padding=1)
        self.conv3 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(64*3*10, 32)
        self.fc2 = nn.Linear(32, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 64*3*10)
        x = self.dropout(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
net = Net()

In [4]:

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

epochs = 15
for epoch in range(epochs):
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs.unsqueeze(1))
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 1.8695447444915771
Epoch: 2, Loss: 1.9746652841567993
Epoch: 3, Loss: 1.9377230405807495
Epoch: 4, Loss: 1.812574863433838
Epoch: 5, Loss: 1.8660504817962646
Epoch: 6, Loss: 1.9063644409179688
Epoch: 7, Loss: 1.9298654794692993
Epoch: 8, Loss: 1.9371302127838135
Epoch: 9, Loss: 1.9142595529556274
Epoch: 10, Loss: 1.8853342533111572
Epoch: 11, Loss: 1.9845359325408936
Epoch: 12, Loss: 1.922088384628296
Epoch: 13, Loss: 1.8595125675201416
Epoch: 14, Loss: 1.9377131462097168
Epoch: 15, Loss: 1.8986506462097168


In [5]:
net.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        outputs = net(inputs.unsqueeze(1))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.long()).sum().item()

print(f'Accuracy: {100 * correct / total}%')

Accuracy: 57.00833333333333%
